In [2]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 python-dotenv==1.0.1 langchain-anthropic langchain-google-genai httpx==0.27.2


INFO: pip is looking at multiple versions of langchain-anthropic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-anthropic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain

In [3]:
import operator
from typing import Annotated, Any, Optional

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

import os
from google.colab import userdata

# 必要に応じてAPIキーやLangChainの設定を行う
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "seo-keyword-generator"

########################################
# データモデル
########################################

# ペルソナを表すデータモデル（検索者の特徴・背景）
class Persona(BaseModel):
    name: str = Field(..., description="検索ユーザーの名前")
    background: str = Field(..., description="検索ユーザーが抱える悩みや状況など")

# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(
        default_factory=list,
        description="想定される検索ユーザーのリスト"
    )

# インタビュー（質問/回答）を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(..., description="インタビュー対象のペルソナ")
    question: str = Field(..., description="インタビュアーからの質問（どんなキーワードで検索するかなど）")
    answer: str = Field(..., description="ペルソナの回答（実際に検索するとしたらどんなキーワードを使うか）")

# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(
        default_factory=list,
        description="すべてのインタビュー結果（ペルソナごとのキーワード候補など）"
    )

########################################
# SEOキーワード選定 エージェント用 State
########################################
class SEOInterviewState(BaseModel):
    user_request: str = Field(..., description="SEOキーワードを考えたいサービス・製品の情報")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list,
        description="生成された検索ユーザーのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list,
        description="インタビュー結果のリスト"
    )
    keyword_doc: str = Field(
        default="",
        description="最終的に生成されるSEOキーワード候補一覧"
    )

########################################
# PersonaGenerator
# (1) どんな検索者（ペルソナ）が想定されるか
########################################
class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 3):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, service_desc: str) -> Personas:
        """
        service_desc を元に、多様な検索者像を k 人分生成
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはSEOのターゲットユーザー像を想定する専門家です。"
                ),
                (
                    "human",
                    f"以下のサービス(または製品)に興味を持ちそうな検索ユーザーを、{self.k}人のペルソナとして生成してください。\n\n"
                    "【サービス概要】\n{service_desc}\n\n"
                    "各ペルソナには：\n"
                    " - 名前\n"
                    " - 年齢・職業などの背景\n"
                    " - サービスを探す理由や悩み\n"
                    "などを具体的に書いてください。\n"
                    "出力は日本語でお願いします。",
                ),
            ]
        )
        chain = prompt | self.llm
        return chain.invoke({"service_desc": service_desc})

########################################
# InterviewConductor
# (2) ペルソナごとに「どんなキーワードで検索するか」を尋ね、回答を得る
########################################
class InterviewConductor:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, service_desc: str, personas: list[Persona]) -> InterviewResult:
        questions = self._generate_questions(service_desc, personas)
        answers = self._generate_answers(personas, questions)
        interviews = self._create_interviews(personas, questions, answers)
        return InterviewResult(interviews=interviews)

    def _generate_questions(self, service_desc: str, personas: list[Persona]) -> list[str]:
        """
        ペルソナに対して、「どんなキーワードで検索するか」を聞き出す質問を生成
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはSEOコンサルタントであり、検索ユーザーの生の声（どう検索するか）を引き出すインタビュアーです。"
                ),
                (
                    "human",
                    "以下のペルソナに対して、『あなたならどんなキーワードで検索エンジンを利用して、このサービスを探しそうですか？』と質問するための文面を1つ作成してください。\n\n"
                    "【サービス概要】\n{service_desc}\n"
                    "【ペルソナ】\n名前：{persona_name}\n背景：{persona_background}\n\n"
                    "質問はシンプルかつオープンエンドな形にしてください。"
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()

        queries = [
            {
                "service_desc": service_desc,
                "persona_name": p.name,
                "persona_background": p.background,
            }
            for p in personas
        ]
        return chain.batch(queries)

    def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
        """
        各ペルソナに対し、「こういうキーワードで検索しそう」と回答を作らせる
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは以下のペルソナ（検索ユーザー）です。インタビュアーに対し、実際に検索するとしたら使いそうなキーワードやフレーズを具体的に挙げてください。"
                ),
                (
                    "human",
                    "【ペルソナ情報】\n名前: {persona_name}\n背景: {persona_background}\n\n質問: {question}"
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()

        queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question,
            }
            for persona, question in zip(personas, questions)
        ]
        return chain.batch(queries)

    def _create_interviews(
        self, personas: list[Persona], questions: list[str], answers: list[str]
    ) -> list[Interview]:
        return [
            Interview(persona=persona, question=q, answer=a)
            for persona, q, a in zip(personas, questions, answers)
        ]

########################################
# KeywordDocumentGenerator
# (3) インタビュー結果をまとめ、SEOキーワード候補リストを作成
########################################
class KeywordDocumentGenerator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, service_desc: str, interviews: list[Interview]) -> str:
        """
        ペルソナごとの回答をもとに、最終的なSEOキーワード候補リストを生成する
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはSEOキーワード選定の専門家です。"
                ),
                (
                    "human",
                    "以下の【サービス概要】と【インタビュー結果】から、ユーザーが検索しそうなキーワード候補一覧をまとめてください。\n"
                    "できるだけ多様な検索意図をカバーするようにキーワード案をリストアップし、日本語で出力してください。\n\n"
                    "【サービス概要】\n{service_desc}\n\n"
                    "【インタビュー結果】\n{interview_results}\n"
                    "それぞれのペルソナが挙げたキーワードを整理・分析し、\n"
                    "最終的な『SEOキーワード候補一覧』として提案してください。\n\n"
                    "キーワードに対する検索意図など、簡単なコメントもあると尚良いです。\n\n"
                    "出力は日本語でお願いします。"
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()

        interview_text = ""
        for i in interviews:
            interview_text += (
                f"▼ペルソナ: {i.persona.name} - {i.persona.background}\n"
                f" 質問: {i.question}\n"
                f" 回答: {i.answer}\n\n"
            )

        return chain.invoke({
            "service_desc": service_desc,
            "interview_results": interview_text
        })

########################################
# SEOキーワード選定エージェント
# ペルソナ生成 → インタビュー → キーワード候補リスト生成
########################################
class SEOKeywordAgent:
    def __init__(self, llm: ChatOpenAI, k: Optional[int] = 3):
        self.persona_generator = PersonaGenerator(llm=llm, k=k)
        self.interview_conductor = InterviewConductor(llm=llm)
        self.keyword_generator = KeywordDocumentGenerator(llm=llm)

        # StateGraphの組み立て
        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        """
        1) ペルソナ生成 → 2) インタビュー実施 → 3) キーワード候補リスト生成 → END
        """
        workflow = StateGraph(SEOInterviewState)

        # ノード登録
        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("generate_keywords", self._generate_keywords)

        # エントリーポイント設定
        workflow.set_entry_point("generate_personas")

        # 遷移定義
        workflow.add_edge("generate_personas", "conduct_interviews")
        workflow.add_edge("conduct_interviews", "generate_keywords")
        workflow.add_edge("generate_keywords", END)

        # コンパイル
        return workflow.compile()

    def _generate_personas(self, state: SEOInterviewState) -> dict[str, Any]:
        # ペルソナを生成
        new_personas: Personas = self.persona_generator.run(state.user_request)
        return {
            "personas": state.personas + new_personas.personas
        }

    def _conduct_interviews(self, state: SEOInterviewState) -> dict[str, Any]:
        # 生成された最後の5人（または state.personas 全体でも可）を対象にインタビュー
        new_personas = state.personas[-5:]
        interviews_result: InterviewResult = self.interview_conductor.run(
            state.user_request, new_personas
        )
        return {
            "interviews": state.interviews + interviews_result.interviews
        }

    def _generate_keywords(self, state: SEOInterviewState) -> dict[str, Any]:
        # インタビュー結果を元にSEOキーワード候補リストを作成
        keyword_doc: str = self.keyword_generator.run(
            state.user_request, state.interviews
        )
        return {"keyword_doc": keyword_doc}

    def run(self, service_desc: str) -> str:
        """
        メイン実行関数：ユーザーが提示するサービス内容を元に最終的な「SEOキーワード候補リスト」を返す
        """
        initial_state = SEOInterviewState(user_request=service_desc)
        final_state = self.graph.invoke(initial_state)
        return final_state["keyword_doc"]


########################################
# main関数（サンプル実行例）
########################################
def main():
    # 1) サービスや製品の概要を入力
    service_desc = input("SEOキーワードを考えたいサービス（製品）について簡単に説明してください: ")

    # 2) Persona生成数
    k = 3  # 必要に応じて変更

    # 3) ChatOpenAIモデルの初期化
    # llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", temperature=0.0)
    llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash-thinking-exp-01-21")
    # llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

    # 4) SEOKeywordAgent の生成
    agent = SEOKeywordAgent(llm=llm, k=k)

    # 5) 実行して最終的なキーワード候補一覧を取得
    final_keywords = agent.run(service_desc=service_desc)

    # 6) 結果の表示
    print("----- 生成されたSEOキーワード候補一覧 -----")
    print(final_keywords)


if __name__ == "__main__":
    main()


SEOキーワードを考えたいサービス（製品）について簡単に説明してください: 東京のフリーランスのSEOコンサルサービス。特徴はAI、Pythonを駆使した業務効率化や、格安での提供、データサイエンスを活用したデータ分析、D2C（ブランド）での豊富な経験などです。


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x78635e113890>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:

{'name': 'Personas', 'parameters': {'type_': 6, 'properties': {'personas': {'type_': 5, 'description': '想定される検索ユーザーのリスト', 'items': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'required': []}, 'description': ''}


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x78635e113890>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:

KeyboardInterrupt: 